In [1]:
# install necessary libraries and dependencies
!pip install sentence_transformers
!pip install scikit_posthocs
from sklearn.utils import resample
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, ttest_rel, wilcoxon, chi2_contingency, fisher_exact, spearmanr
from statsmodels.stats.proportion import proportions_ztest
import scikit_posthocs as sp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.3 MB/s eta 0:00:00


In [2]:
# load distilgpt model
distilgpt_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
distilgpt_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
distilgpt_tokenizer.pad_token = distilgpt_tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
# load gpt-2 model
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
# Define the models and tokenizers dictionaries
models = {
    'DistilGPT': distilgpt_model,
    'GPT-2': gpt2_model
}

tokenizers = {
    'DistilGPT': distilgpt_tokenizer,
    'GPT-2': gpt2_tokenizer
}

In [5]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distilgpt_model.to(device)
gpt2_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
# load the sampled data
sampled_df = pd.read_csv('/content/drive/My Drive/ITNPBD5 PROJECT/sampled_empathetic_dialogues.csv')

We create our functions for calculating perplexity and lexical similarity scores

In [7]:
# load sentence transformer for lexical similarity
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# function for perplexity function
def calculate_perplexity(model, tokenizer, text):
    try:
        encodings = tokenizer(text, return_tensors='pt').to(device)
        max_length = model.config.max_position_embeddings
        stride = 512
        seq_len = encodings.input_ids.size(1)

        nlls = []
        prev_end_loc = 0
        for begin_loc in range(0, seq_len, stride):
            end_loc = min(begin_loc + max_length, seq_len)
            trg_len = end_loc - prev_end_loc
            input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
            target_ids = input_ids.clone()
            target_ids[:, :-trg_len] = -100

            with torch.no_grad():
                outputs = model(input_ids, labels=target_ids)
                neg_log_likelihood = outputs.loss * trg_len

            nlls.append(neg_log_likelihood)
            prev_end_loc = end_loc
            if end_loc == seq_len:
                break

        if len(nlls) > 0:
            ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
            return min(ppl.item(), 1e6)  # Cap at a large but finite value
        else:
            return 1e6  # a large but finite value for empty responses
    except Exception as e:
        print(f"Error calculating perplexity: {e}")
        return 1e6  # a large but finite value for errors

In [9]:
# funtion for similarity score
def calculate_lexical_similarity(text1, text2):
    try:
        embeddings = sentence_model.encode([text1, text2])
        similarity = cosine(embeddings[0], embeddings[1])
        return max(0, min(similarity, 1))  # Ensure it's between 0 and 1
    except Exception as e:
        print(f"Error calculating similarity: {e}")
        return 0  # minimum similarity for errors

In [10]:
# set similarity threshold
SIMILARITY_THRESHOLD = 0.05

# function for valid responses
def is_valid_response(response, min_words=3, max_words=50):
    words = response.split()
    return min_words <= len(words) <= max_words and not any(char in response for char in "[](){}")

We create our functions for response generation and structure for the zero-shot generation which includes the calculations for perplexity and similarity scores

In [11]:
# function for response generation
def generate_response(model, tokenizer, input_text, max_new_tokens=50,):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [12]:
#function for the zero shot experiment
def zero_shot_experiment(model, tokenizer, df, num_generations=3):
    results = []
    empty_response_count = 0
    samples_with_valid_responses = 0

    for _, row in tqdm(df.iterrows(), total=len(df)):
        context = row['context']
        prompt = row['prompt']
        input_text = row['input']
        target = row['target']

        zero_shot_prompt = f"""Context: In this situation, the person is feeling {context}.
        Conversation: {input_text}
        Task: Generate an empathetic response that acknowledges their feelings and offers support.
        Empathetic response:"""

        valid_responses = []
        perplexities = []
        similarities = []

        for _ in range(num_generations):
            generated_response = generate_response(model, tokenizer, zero_shot_prompt)
            clean_response = generated_response.split("Empathetic response:")[-1].strip()

            if clean_response and is_valid_response(clean_response):
                similarity = calculate_lexical_similarity(target, clean_response)
                perplexity = calculate_perplexity(model, tokenizer, clean_response)

                if np.isfinite(similarity) and np.isfinite(perplexity):
                    valid_responses.append(clean_response)
                    perplexities.append(perplexity)
                    similarities.append(similarity)
            else:
                empty_response_count += 1

        if valid_responses:
            samples_with_valid_responses += 1
            avg_perplexity = np.mean(perplexities)
            avg_similarity = np.mean(similarities)
            best_response_idx = np.argmax(similarities)
            best_response = valid_responses[best_response_idx]
        else:
            avg_perplexity = 1e6
            avg_similarity = 0
            best_response = ""

        results.append({
            'context': context,
            'prompt': prompt,
            'input': input_text,
            'target': target,
            'generated_response': best_response,
            'avg_perplexity': avg_perplexity,
            'avg_lexical_similarity': avg_similarity,
            'num_valid_responses': len(valid_responses),
            'is_relevant': avg_similarity > SIMILARITY_THRESHOLD
        })

    print(f"Total empty responses: {empty_response_count}")
    print(f"Samples with at least one valid response: {samples_with_valid_responses}/{len(df)}")
    return pd.DataFrame(results)

### DistilGPT Generation for the whole sample

In [ ]:
# run distilgpt experiment
print("Starting zero-shot experiment with DistilGPT...")
distilgpt_results = zero_shot_experiment(distilgpt_model, distilgpt_tokenizer, sampled_df)
distilgpt_results['model'] = 'DistilGPT'

Starting zero-shot experiment with DistilGPT...


100%|██████████| 1000/1000 [15:49<00:00,  1.05it/s]

Total empty responses: 121
Samples with at least one valid response: 1000/1000


### GPT-2 generation for the whole sample

In [ ]:
# run gpt-2 experiment
print("Starting zero-shot experiment with GPT-2...")
gpt2_results = zero_shot_experiment(gpt2_model, gpt2_tokenizer, sampled_df)
gpt2_results['model'] = 'GPT-2'

Starting zero-shot experiment with GPT-2...


100%|██████████| 1000/1000 [27:31<00:00,  1.65s/it]

Total empty responses: 291
Samples with at least one valid response: 1000/1000


In [ ]:
# save results
distilgpt_results.to_csv('/content/drive/My Drive/ITNPBD5 PROJECT/distilgpt_zero_shot_results.csv', index=False)
gpt2_results.to_csv('/content/drive/My Drive/ITNPBD5 PROJECT/gpt2_zero_shot_results.csv', index=False)

In [ ]:
# combine results
combined_results = pd.concat([distilgpt_results, gpt2_results], ignore_index=True)

In [ ]:
# save combined results
combined_results.to_csv('/content/drive/My Drive/ITNPBD5 PROJECT/zero_shot_combined_results.csv', index=False)
print("Combined one-shot results saved.")

Combined one-shot results saved.


In [ ]:
#print model statistics
def print_model_statistics(df, model_name):
    print(f"\n--- {model_name} Statistics ---")
    print(f"Total samples: {len(df)}")
    print(f"Valid responses: {df['num_valid_responses'].sum()}")
    print(f"Samples with at least one valid response: {(df['num_valid_responses'] > 0).sum()}")

    print("\nPerplexity:")
    print(f" Mean: {df['avg_perplexity'].mean():.4f}")
    print(f" Median: {df['avg_perplexity'].median():.4f}")
    print(f" Std Dev: {df['avg_perplexity'].std():.4f}")
    print(f" Min: {df['avg_perplexity'].min():.4f}")
    print(f" Max: {df['avg_perplexity'].max():.4f}")

    print("\nLexical Similarity:")
    print(f" Mean: {df['avg_lexical_similarity'].mean():.4f}")
    print(f" Median: {df['avg_lexical_similarity'].median():.4f}")
    print(f" Std Dev: {df['avg_lexical_similarity'].std():.4f}")
    print(f" Min: {df['avg_lexical_similarity'].min():.4f}")
    print(f" Max: {df['avg_lexical_similarity'].max():.4f}")

    print("\nZero Similarity Scores:")
    zero_sim = (df['avg_lexical_similarity'] == 0).sum()
    print(f" Count: {zero_sim}")
    print(f" Percentage: {zero_sim / len(df) * 100:.2f}%")

print_model_statistics(distilgpt_results, "DistilGPT")
print_model_statistics(gpt2_results, "GPT-2")


--- DistilGPT Statistics ---
Total samples: 1000
Valid responses: 2879
Samples with at least one valid response: 1000

Perplexity:
 Mean: 21.2545
 Median: 18.8874
 Std Dev: 13.6839
 Min: 10.2541
 Max: 259.0756

Lexical Similarity:
 Mean: 0.8646
 Median: 0.8744
 Std Dev: 0.0808
 Min: 0.5039
 Max: 1.0000

Zero Similarity Scores:
 Count: 0
 Percentage: 0.00%

--- GPT-2 Statistics ---
Total samples: 1000
Valid responses: 2709
Samples with at least one valid response: 1000

Perplexity:
 Mean: 15.6499
 Median: 15.0573
 Std Dev: 3.6914
 Min: 6.1182
 Max: 46.3152

Lexical Similarity:
 Mean: 0.8974
 Median: 0.9136
 Std Dev: 0.0801
 Min: 0.5187
 Max: 1.0000

Zero Similarity Scores:
 Count: 0
 Percentage: 0.00%


In [ ]:
# print model performance
def report_model_performance(df, model_name):
    total_samples = len(df)
    relevant_responses = df['is_relevant'].sum()
    relevant_df = df[df['is_relevant']]

    print(f"{model_name} Performance:")
    print(f"Total samples: {total_samples}")
    print(f"Relevant responses: {relevant_responses} ({relevant_responses/total_samples*100:.2f}%)")
    print(f"Average similarity (relevant only): {relevant_df['avg_lexical_similarity'].mean():.4f}")
    print(f"Average perplexity (relevant only): {relevant_df['avg_perplexity'].mean():.4f}")

    similarity_bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]
    similarity_distribution = pd.cut(df['avg_lexical_similarity'], bins=similarity_bins).value_counts().sort_index()
    print("\nSimilarity Distribution:")
    print(similarity_distribution)
    print("---")

report_model_performance(distilgpt_results, "DistilGPT")
report_model_performance(gpt2_results, "GPT-2")

DistilGPT Performance:
Total samples: 1000
Relevant responses: 1000 (100.00%)
Average similarity (relevant only): 0.8646
Average perplexity (relevant only): 21.2545

Similarity Distribution:
avg_lexical_similarity
(0.0, 0.1]       0
(0.1, 0.2]       0
(0.2, 0.3]       0
(0.3, 0.4]       0
(0.4, 0.5]       0
(0.5, 1.0]    1000
Name: count, dtype: int64
---
GPT-2 Performance:
Total samples: 1000
Relevant responses: 1000 (100.00%)
Average similarity (relevant only): 0.8974
Average perplexity (relevant only): 15.6499

Similarity Distribution:
avg_lexical_similarity
(0.0, 0.1]       0
(0.1, 0.2]       0
(0.2, 0.3]       0
(0.3, 0.4]       0
(0.4, 0.5]       0
(0.5, 1.0]    1000
Name: count, dtype: int64
---


To improve our analysis and , we include a count of how many samples had at least one valid response for each model, and a report on the distribution of valid responses per sample (e.g., how many samples had 1, 2, or 3 valid responses).

In [ ]:
# check number and percenatge of empty responses
total_samples = len(sampled_df)
distilgpt_empty = total_samples * 3 - distilgpt_results['num_valid_responses'].sum()
gpt2_empty = total_samples * 3 - gpt2_results['num_valid_responses'].sum()

print(f"DistilGPT empty responses: {distilgpt_empty}/{total_samples * 3} ({distilgpt_empty / (total_samples * 3) * 100:.2f}%)")
print(f"GPT-2 empty responses: {gpt2_empty}/{total_samples * 3} ({gpt2_empty / (total_samples * 3) * 100:.2f}%)")

DistilGPT empty responses: 121/3000 (4.03%)
GPT-2 empty responses: 291/3000 (9.70%)


In [ ]:
# check the count of valid and distributions of responses
for model_name in ['DistilGPT', 'GPT-2']:
    model_results = combined_results[combined_results['model'] == model_name]
    response_distribution = model_results['num_valid_responses'].value_counts().sort_index()
    print(f"\n{model_name} valid response distribution:")
    print(response_distribution)
    print(f"Samples with at least one valid response: {(model_results['num_valid_responses'] > 0).sum()}/{len(model_results)}")


DistilGPT valid response distribution:
num_valid_responses
1      2
2    117
3    881
Name: count, dtype: int64
Samples with at least one valid response: 1000/1000

GPT-2 valid response distribution:
num_valid_responses
1     24
2    243
3    733
Name: count, dtype: int64
Samples with at least one valid response: 1000/1000


### Sample generation
A random sample is chosen and ran through the experiment ten times and the results are aggregated and the best result is printed out

In [ ]:
# Use this sample index to get the same sample from your dataset
specific_sample = sampled_df.iloc[sample_index]

# save the sample data on a file
sample_index_file = '/content/drive/My Drive/sample_index.json'
with open(sample_index_file, 'r') as f:
    sample_index = json.load(f)

In [ ]:
# function for ten runs
def run_multiple_experiments(df, models, tokenizers, specific_sample, num_runs=10):
    all_results = []

    for run in range(num_runs):
        result = zero_shot_generation(df, models, tokenizers, specific_sample)
        all_results.append(result)

    return all_results

In [ ]:
# function for generation
def zero_shot_generation(df, models, tokenizers, specific_sample):
    results = []

    prompt = f"""Context: In this situation, the person is feeling {specific_sample['context']}.
    Conversation: {specific_sample['input']}
    Task: Generate an empathetic response that acknowledges their feelings and offers support.
    Empathetic response:"""

    best_responses = {model_name: None for model_name in models}
    best_metrics = {model_name: {'perplexity': float('inf'), 'similarity': 0} for model_name in models}

    for _ in range(3):  # 3 attempts
        for model_name, model in models.items():
            tokenizer = tokenizers[model_name]
            response = generate_response(model, tokenizer, prompt)
            perplexity = min(calculate_perplexity(model, tokenizer, response), 1e6)  # Cap perplexity at 1e6
            similarity = calculate_lexical_similarity(specific_sample['target'], response)

            if similarity > best_metrics[model_name]['similarity'] or (
                similarity == best_metrics[model_name]['similarity'] and
                perplexity < best_metrics[model_name]['perplexity']
            ):
                best_responses[model_name] = response
                best_metrics[model_name]['perplexity'] = perplexity
                best_metrics[model_name]['similarity'] = similarity

    results.append({
        'sample': specific_sample,
        'responses': best_responses,
        'metrics': best_metrics
    })

    return results

In [ ]:
# function for aggregating results after ten runs
def aggregate_results(all_results):
    aggregated = {model: {'perplexity': [], 'similarity': []} for model in models}
    best_responses = {model: {'response': '', 'perplexity': float('inf'), 'similarity': 0} for model in models}

    for result_list in all_results: # Iterate over the list of results
        for result in result_list: # Iterate over each result dictionary within the list
            for model in models:
                perplexity = result['metrics'][model]['perplexity']
                similarity = result['metrics'][model]['similarity']

                aggregated[model]['perplexity'].append(perplexity)
                aggregated[model]['similarity'].append(similarity)

                # update best response if current one is better
                if similarity > best_responses[model]['similarity'] or \
                   (similarity == best_responses[model]['similarity'] and perplexity < best_responses[model]['perplexity']):
                    best_responses[model] = {
                        'response': result['responses'][model],
                        'perplexity': perplexity,
                        'similarity': similarity
                    }

    for model in models:
        print(f"\n{model} Statistics (over {len(all_results)} runs):")
        print(f"Perplexity - Mean: {np.mean(aggregated[model]['perplexity']):.4f}")
        print(f"Similarity - Mean: {np.mean(aggregated[model]['similarity']):.4f}")
        print("\nBest Generated Response:")
        print(best_responses[model]['response'])
        print(f"Best Response Metrics - Perplexity: {best_responses[model]['perplexity']:.4f}, "
              f"Similarity: {best_responses[model]['similarity']:.4f}")
        print()

# run the experiments
num_runs = 10
all_results = run_multiple_experiments(sampled_df, models, tokenizers, specific_sample, num_runs)

# print sample for generation
print("Sample for generation:")
print(f"Context: {specific_sample['context']}")
print(f"Conversation: {specific_sample['input']}")
print(f"Actual response: {specific_sample['target']}")

# print aggregated results and best responses
aggregate_results(all_results)

Sample for generation:
Context: nostalgic
Conversation: i was at the mall the other day and i smelled some cinnamon rolls. it reminded me of my childhood and was such a good time! oh that cinnabon smell can be so enticing!
Actual response: it was! i bought 2!

DistilGPT Statistics (over 10 runs):
Perplexity - Mean: 27.9731
Similarity - Mean: 0.9341

Best Generated Response:
Context: In this situation, the person is feeling nostalgic.
    Conversation: i was at the mall the other day and i smelled some cinnamon rolls. it reminded me of my childhood and was such a good time! oh that cinnabon smell can be so enticing!
    Task: Generate an empathetic response that acknowledges their feelings and offers support.
    Empathetic response: If the recipient wants to help you out, you can do that and help them out. so they can become more open to you. you don't need to be afraid. do not be scared to ask your friend for help! just help with the
Best Response Metrics - Perplexity: 27.2229, Simila

## Statistical Test

In [ ]:
# load the results
distilgpt_results = pd.read_csv('/content/drive/My Drive/ITNPBD5 PROJECT/distilgpt_zero_shot_results.csv')
gpt2_results = pd.read_csv('/content/drive/My Drive/ITNPBD5 PROJECT/gpt2_zero_shot_results.csv')

In [ ]:
# load the data
data = pd.read_csv('/content/drive/MyDrive/ITNPBD5 PROJECT/zero_shot_combined_results.csv')
data['pair_id'] = data.groupby(['context', 'prompt', 'input', 'target']).ngroup()
valid_samples = data[data['num_valid_responses'] > 0].groupby('pair_id').filter(lambda x: len(x) == 2)
distilgpt_data = valid_samples[valid_samples['model'] == 'DistilGPT']
gpt2_data = valid_samples[valid_samples['model'] == 'GPT-2']

In [ ]:
# ensure data is aligned
distilgpt_data = distilgpt_data.reset_index(drop=True)
gpt2_data = gpt2_data.reset_index(drop=True)

In [ ]:
gpt2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   context                 1000 non-null   object 
 1   prompt                  1000 non-null   object 
 2   input                   1000 non-null   object 
 3   target                  1000 non-null   object 
 4   generated_response      1000 non-null   object 
 5   avg_perplexity          1000 non-null   float64
 6   avg_lexical_similarity  1000 non-null   float64
 7   num_valid_responses     1000 non-null   int64  
 8   is_relevant             1000 non-null   bool   
 9   model                   1000 non-null   object 
 10  pair_id                 1000 non-null   int64  
dtypes: bool(1), float64(2), int64(2), object(6)
memory usage: 79.2+ KB


In [ ]:
# get descriptive statistics from both models
def print_descriptive_stats(data, model_name):
    for metric in ['avg_perplexity', 'avg_lexical_similarity']:
        stats = data[metric].agg(['mean', 'median', 'std', 'min', 'max'])
        print(f"{model_name} {metric}:")
        for stat, value in stats.items():
            print(f"  {stat}: {value:.4f}")
        print()

print_descriptive_stats(distilgpt_data, "DistilGPT")
print_descriptive_stats(gpt2_data, "GPT-2")

DistilGPT avg_perplexity:
  mean: 21.2545
  median: 18.8874
  std: 13.6839
  min: 10.2541
  max: 259.0756

DistilGPT avg_lexical_similarity:
  mean: 0.8646
  median: 0.8744
  std: 0.0808
  min: 0.5039
  max: 1.0000

GPT-2 avg_perplexity:
  mean: 15.6499
  median: 15.0573
  std: 3.6914
  min: 6.1182
  max: 46.3152

GPT-2 avg_lexical_similarity:
  mean: 0.8974
  median: 0.9136
  std: 0.0801
  min: 0.5187
  max: 1.0000



In [ ]:
# hypothesis tests experiment
def paired_hypothesis_test(distilgpt_data, gpt2_data, metric):
    print(f"\nHypothesis Test for {metric.capitalize()}:")
    print(f"Null Hypothesis (H0): There is no difference in mean {metric} between DistilGPT and GPT-2 small.")
    print(f"Alternative Hypothesis (H1): There is a difference in mean {metric} between DistilGPT and GPT-2 small.")

    alpha = 0.05
    print(f"Significance level: {alpha}")

    differences = distilgpt_data[metric] - gpt2_data[metric]

    # normality test
    try:
        _, p_value_normal = stats.normaltest(differences)
        print(f"Normality test p-value: {p_value_normal:.4f}")
    except Exception as e:
        print(f"Error in normality test: {e}")

    # paired t-test
    try:
        t_statistic, t_p_value = stats.ttest_rel(distilgpt_data[metric], gpt2_data[metric])
        print(f"Paired t-test - t-statistic: {t_statistic:.4f}, p-value: {t_p_value:.4f}")
    except Exception as e:
        print(f"Error in paired t-test: {e}")

    # wilcoxon signed-rank test
    try:
        w_statistic, w_p_value = stats.wilcoxon(differences)
        print(f"Wilcoxon test - statistic: {w_statistic:.4f}, p-value: {w_p_value:.4f}")
    except Exception as e:
        print(f"Error in Wilcoxon test: {e}")

    # effect size calculation
    try:
        effect_size = (np.mean(differences) / np.std(differences, ddof=1))
        print(f"Effect size (Cohen's d): {effect_size:.4f}")
    except Exception as e:
        print(f"Error in effect size calculation: {e}")

    # confidence interval
    try:
        ci = stats.t.interval(confidence=0.95, df=len(differences)-1,
                              loc=np.mean(differences),
                              scale=stats.sem(differences))
        print(f"95% Confidence Interval: ({ci[0]:.4f}, {ci[1]:.4f})")
    except Exception as e:
        print(f"Error in confidence interval calculation: {e}")

    # decision and interpretation
    if w_p_value < alpha:  # Using Wilcoxon test p-value for decision
        decision = "Reject the null hypothesis"
        conclusion = f"There is a statistically significant difference in {metric} between DistilGPT and GPT-2 small."
    else:
        decision = "Fail to reject the null hypothesis"
        conclusion = f"There is not enough evidence to conclude a significant difference in {metric} between DistilGPT and GPT-2 small."

    print(f"\nDecision: {decision}")
    print(f"Conclusion: {conclusion}")

    if abs(effect_size) < 0.2:
        effect_size_interpretation = "small"
    elif abs(effect_size) < 0.5:
        effect_size_interpretation = "medium"
    else:
        effect_size_interpretation = "large"

    print(f"\nThe effect size is {effect_size_interpretation} (Cohen's d = {effect_size:.4f}).")

    if ci[0] * ci[1] > 0:
        ci_interpretation = f"favors {'DistilGPT' if ci[0] > 0 else 'GPT-2'}"
    else:
        ci_interpretation = "includes zero, indicating uncertainty in the direction of the effect"

    print(f"The 95% confidence interval {ci_interpretation}.")

# run the tests
paired_hypothesis_test(distilgpt_data, gpt2_data, 'avg_perplexity')
paired_hypothesis_test(distilgpt_data, gpt2_data, 'avg_lexical_similarity')


Hypothesis Test for Avg_perplexity:
Null Hypothesis (H0): There is no difference in mean avg_perplexity between DistilGPT and GPT-2 small.
Alternative Hypothesis (H1): There is a difference in mean avg_perplexity between DistilGPT and GPT-2 small.
Significance level: 0.05
Normality test p-value: 0.0000
Paired t-test - t-statistic: 12.4823, p-value: 0.0000
Wilcoxon test - statistic: 72807.0000, p-value: 0.0000
Effect size (Cohen's d): 0.3947
95% Confidence Interval: (4.7235, 6.4857)

Decision: Reject the null hypothesis
Conclusion: There is a statistically significant difference in avg_perplexity between DistilGPT and GPT-2 small.

The effect size is medium (Cohen's d = 0.3947).
The 95% confidence interval favors DistilGPT.

Hypothesis Test for Avg_lexical_similarity:
Null Hypothesis (H0): There is no difference in mean avg_lexical_similarity between DistilGPT and GPT-2 small.
Alternative Hypothesis (H1): There is a difference in mean avg_lexical_similarity between DistilGPT and GPT-2 